In [224]:
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

MAX_EVENT_DISPLAY = 15 # custom number of neighbors to display
TEAM_NODE_A = 1740 # home team node to focus on and see neighbors of
TEAM_NODE_B = 1455 # away team node to focus on and see neighbors of
FIG_SIZE = 10 # graph display size

In [220]:
df = pd.read_csv('data/processed/lol/events.csv')

In [228]:
focusNodes = [
    {
        'name': TEAM_NODE_A,
        'nodeList': [],
        'edgeList': []
    },
    {
        'name': TEAM_NODE_B,
        'nodeList': [],
        'edgeList': []
    }
]

nodeLabelMapper = {
    1: '  Team:  \n',
    2: '  Player:  \n'
}

edgeLabelMapper = {
    1: 'Lost',
    2: 'Won',
    3: 'Joined',
    4: 'Info'
}

# Keys the same as node type
# (1) team
# (2) player
nodeSizeMapper = {
    1: 30,
    2: 15
}

nodeColorMapper = {
    1: '#000099',
    2: '#0099FF'
}

# Keys the same as edge_type
# (1) Lost
# (2) Won
# (3) Played
# (4) Game info
edgeColorMapper = {
    1: '#FF0000',
    2: '#15B01A',
    3: '#0099FF',
    4: '#C0C0C0'
}

edgeFontSizeMapper = {
    1: 24,
    2: 24,
    3: 12,
    4: 10
}

for focusNode in focusNodes:
    selectedTeam = df.loc[df['u'] == focusNode['name'], ['u', 'v', 'u_type', 'v_type', 'e_type']].head(MAX_EVENT_DISPLAY)
    teamTypeDF = selectedTeam.drop_duplicates()
    # print(teamTypeDF)
    us = list(teamTypeDF[['u', 'u_type']].itertuples(index=False, name=None))
    vs = list(teamTypeDF[['v', 'v_type']].itertuples(index=False, name=None))
    nodesList = list(list(dict.fromkeys(us + vs)))
    nodesList = list(map(lambda x: (x[0], {
        'type': x[1],
        'label': nodeLabelMapper[x[1]] + str(x[0]),
        'title': 'TODO',
        'margin': 15,
        'color': {
            'background': nodeColorMapper[x[1]], 
            'highlight': {
                'background': nodeColorMapper[x[1]],
                'border': 'magenta'
            },
            'hover': {
                'border': 'gray'
            }
        },
        # 'physics': False,
        # 'mass': 8 if x[1] == 1 else 2,
        'mass': 5,
        'shape': 'circle',
        'font': {'size': 30 if x[1] == 1 else 18, 'color': 'white'},
        'borderWidthSelected': 6
        }), nodesList))
    edgesWithDups = selectedTeam.groupby(selectedTeam.columns.tolist(), as_index=False).size()
    edgesWithDupsList = list(edgesWithDups[['u', 'v', 'e_type', 'size']].itertuples(index=False, name=None))
    # Change arrow display direction if edge type is 3 (to show player joining team):
    edgesWithDupsList = list(map(lambda x: (x[1] if x[2] == 3 else x[0], x[0] if x[2] == 3 else x[1], { 
        'edge_type': x[2], 
        'weight': x[3] * 3,
        'label': edgeLabelMapper[x[2]],
        'title': 'EDGE',
        'color': edgeColorMapper[x[2]],
        'font': {'size': edgeFontSizeMapper[x[2]]},
        'smooth': True
        # 'arrowSize': 5,
        # 'physics': False
        }), edgesWithDupsList))
    focusNode['nodeList'] = nodesList
    focusNode['edgeList'] = edgesWithDupsList
    # print('============')
    # print(focusNode['name'])
    # print(focusNode['nodeList'])
    # for edgeMe in focusNode['edgeList']:
    #     if (edgeMe[0] in [1740, 1455] and edgeMe[1] in [1740, 1455]):
    #         print(edgeMe)
    # print('============')
    # print(edgesWithDupsList)







focusGraph = nx.MultiDiGraph()

lineWidthMapper = {
    1: 6,
    2: 6,
    3: 3,
    4: 1
}

arrowSizeMapper = {
    1: 10,
    2: 40,
    3: 12,
    4: 1
}

allNodesList = focusNodes[0]['nodeList'] + focusNodes[1]['nodeList']
allEdgesList = focusNodes[0]['edgeList'] + focusNodes[1]['edgeList']

focusGraph.add_nodes_from(allNodesList)
focusGraph.add_edges_from(allEdgesList)

options = {
    'arrowstyle': '->',
    'arrowsize': list(arrowSizeMapper[edge_type] for u, v, edge_type in list(focusGraph.edges(data='edge_type')))
}

from pyvis.network import Network
import pyvis

net = Network(
    '1500px', '1500px',
    directed=True,
    # heading='League of Legends',
    select_menu=True,
    filter_menu=True,
    # bgcolor='#222222',
    # font_color='white'
)
net.from_nx(focusGraph) # Create directly from nx graph

options = {
    'physics':{
        'barnesHut':{
            'gravitationalConstant':-50000,
            'centralGravity': 2,
            'springLength': 20,
            'springConstant': 0.7,
            'damping': 3,
            'avoidOverlap': 10
        }
    },
    'interaction':{   
        'selectConnectedEdges': True,
        'hover': True
    },
    'edges': {
        'arrowStrikethrough': False
    }
}

net.options=options

net.show('test1.html', notebook=False) # do NOT remove the notebook=False

           u     v  u_type  v_type  e_type
744453  1740  2627       1       2       3
744454  1740  5002       1       2       3
744455  1740  6746       1       2       3
744456  1740  9932       1       2       3
744460  1740  3493       1       2       3
744462  1740  1455       1       1       1
744467  1740  9932       1       2       4
744468  1740  2627       1       2       4
744469  1740  3493       1       2       4
744470  1740  1455       1       1       4
744472  1740  6746       1       2       4
744473  1740  5002       1       2       4
           u      v  u_type  v_type  e_type
741787  1455   9252       1       2       3
741788  1455   2572       1       2       3
741789  1455   4850       1       2       3
741791  1455  10240       1       2       3
741796  1455   5179       1       2       3
741802  1455   7240       1       2       3
741803  1455   6830       1       2       3
741804  1455  10080       1       2       3
741805  1455   4600       1       2       3
7

In [222]:
# focusGraph = nx.MultiDiGraph()

# lineWidthMapper = {
#     1: 6,
#     2: 6,
#     3: 3,
#     4: 1
# }

# arrowSizeMapper = {
#     1: 10,
#     2: 40,
#     3: 12,
#     4: 1
# }

# allNodesList = focusNodes[0]['nodeList'] + focusNodes[1]['nodeList']
# allEdgesList = focusNodes[0]['edgeList'] + focusNodes[1]['edgeList']

# focusGraph.add_nodes_from(allNodesList)
# focusGraph.add_edges_from(allEdgesList)

# options = {
#     'arrowstyle': '->',
#     'arrowsize': list(arrowSizeMapper[edge_type] for u, v, edge_type in list(focusGraph.edges(data='edge_type')))
# }

# fig = plt.figure(figsize=(FIG_SIZE, FIG_SIZE))
# pos = nx.spring_layout(focusGraph)

# nx.draw_networkx(
#     focusGraph,
#     pos=pos,
#     connectionstyle='arc3, rad = 0.1',
#     arrows=True,
#     **options,
#     with_labels=True,
#     # node_color=list(nodeColorMapper[type] for u, type in list(focusGraph.nodes(data='type'))),
#     # edge_color=list(edgeColorMapper[edge_type] for u, v, edge_type in list(focusGraph.edges(data='edge_type'))),
#     # node_size=list(nodeSizeMapper[type] for u, type in list(focusGraph.nodes(data='type'))),
#     font_color="white",
#     font_size=8,
#     font_family="Tahoma",
#     font_weight="bold",
#     width=list(lineWidthMapper[edge_type] for u, v, edge_type in list(focusGraph.edges(data='edge_type')))
# )

# To add an invidual/multiple edges separately:
# nx.draw_networkx_edges(
#     focusGraph, 
#     pos, 
#     arrows=True,
#     connectionstyle='arc3, rad = 0.1',
#     edgelist=[(1455, 1740)], 
#     width=3)

# plt.margins(0.2)


# plt.show()

# from pyvis.network import Network

# net = Network()

# net.add_node(1, label='Alex')
# net.add_node(2, label='Cathy')

# net.show('nodes.html', notebook=False) # do NOT remove the notebook=False

In [223]:
# from pyvis.network import Network

# net = Network(
#     '1500px', '1500px',
#     directed=True
# )
# net.from_nx(focusGraph) # Create directly from nx graph

# net.show('test1.html', notebook=False) # do NOT remove the notebook=False